# Data Science I - Projeto final

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
df = pd.read_csv('titanic-data-6.csv')

## Perguntas (Questions)

    1. Quais foram os fatores que fizeram com que algumas pessoas fossem mais propensas a sobreviver?
    2. Quais são as características comuns entre os passageiros da mesma classe? 
    3. Em qual classe havia mais familiares?
    4. Qual a proporção de tripulantes e passageiros que sobreviveram?

## Preparar dados (Wrangle data)

In [3]:
df.shape

(891, 12)

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [6]:
# Verificar o número de gêneros cadastrado
df['Sex'].nunique()

2

In [7]:
# Verificar se existe campo com valor nulo
df.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [8]:
# Preencher valores nulos de Idade com a média
df['Age'].fillna(round(df['Age'].mean()), inplace=True)

In [9]:
# Verificar se há passageiros não embarcados marcados como não sobreviventes.
df[df['Survived'] == 0]['Embarked'].isnull().any().sum()

0

In [10]:
# Remover passageiros sem local de embarque
df['Embarked'].dropna(how='any', inplace=True)

In [11]:
# Converter informação de sobrevivente para Boolean
df['Survived'] = df['Survived'].apply(lambda value: bool(value))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,True,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,False,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
# Remover a coluna de Ticket, não agrega valor
df.drop(labels='Ticket', axis=1, inplace=True)

In [13]:
# Remover coluna de PassengerId
df.drop(labels='PassengerId', axis=1, inplace=True)

In [14]:
# Listar colunas
df.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

## Explorar (Explore)

In [ ]:
pd.plotting.scatter_matrix(df[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']], figsize=(15,15));

In [ ]:
df.hist(figsize=(12,12));

In [ ]:
df_survived_class = df.groupby(['Pclass', 'Survived']).size().to_frame(name='Count').reset_index()
df[['Pclass','Survived']]

In [ ]:
# total de pessoas a bordo
total_people_in = df.shape[0]
class_1_survive = df_survive_class.query('Survived == "1"')[['Pclass','count']][]
type(class_1_survive)
# survive_class_total = 
# print('Total de pessoas', df.size[1])
# df_survive_class.query('Survived == "1"')[['Pclass','count']]


In [ ]:
# Contar sobreviventes ou não em cada classe
df.groupby('Pclass')['Survived'].value_counts();